In [74]:
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as f
from isodate import parse_duration
import pyspark.sql.types as t
from typing import Tuple
import os

In [27]:
@f.udf(returnType=t.IntegerType())
def converter_segundos(tempo: str) -> int:
    """Converte para o total de seguntos

    Args:
        tempo (str): tempo

    Returns:
        int: total segundos
    """
    duracao = parse_duration(tempo)
    total_minutes = duracao.total_seconds() 
   
    return int(total_minutes)


In [28]:
spark = SparkSession.builder.appName('tratamento_camada_ouro').getOrCreate()
spark

In [29]:
def carregar_dataframe_prata(spark_session : SparkSession, assunto: str, metrica: str,):
    df_cities_skylines = spark_session \
        .read \
            .parquet(f'/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/prata/{assunto}/extracao_data_*/{metrica}/*.parquet')
    return df_cities_skylines

In [78]:
def criar_particao_ouro(dataframe: DataFrame, assunto_pesquisa: str, metrica: str, colunas_particao: Tuple, nome_arquivo: str):
    caminho_datalake_ouro = '/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/ouro/'
    dataframe.write. \
        partitionBy(*colunas_particao). \
        parquet(os.path.join(caminho_datalake_ouro, assunto_pesquisa, metrica, nome_arquivo))


In [30]:
def transformacao_dataframe(dataframe_transform: DataFrame):
    dataframe_transform = dataframe_transform.select(
        'data_extracao',
        f.explode('DATA_PUBLICACAO').alias('DATA_PUBLICACAO'),
        'ID_CANAL',
        'NM_CANAL',
        'ID_CATEGORIA',
        'ID_VIDEO',
        'TITULO_VIDEO',
        'DESCRICAO',
        'TAGS',
        'DURACAO_VIDEOS',
        'TOTAL_VISUALIZACOES',
        'TOTAL_LIKES',
        'TOTAL_FAVORITOS',
        'TOTAL_COMENTARIOS',
        
     ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                f.explode('ID_CANAL').alias('ID_CANAL'),
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                f.explode('NM_CANAL').alias('NM_CANAL'), 
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                f.explode('ID_CATEGORIA').alias('ID_CATEGORIA'),
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                f.explode('ID_VIDEO').alias('ID_VIDEO'),
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                f.explode('TITULO_VIDEO').alias('TITULO_VIDEO'),
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'TITULO_VIDEO',
                'ID_VIDEO',
                f.explode('DESCRICAO').alias('DESCRICAO'),
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                f.explode('TAGS').alias('TAGS'),
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                f.explode('DURACAO_VIDEOS').alias('DURACAO_VIDEOS'),
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                f.explode('TOTAL_VISUALIZACOES').alias('TOTAL_VISUALIZACOES'),
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                f.explode('TOTAL_LIKES').alias('TOTAL_LIKES'),
                'TOTAL_FAVORITOS',
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                f.explode('TOTAL_FAVORITOS').alias('TOTAL_FAVORITOS'),
                'TOTAL_COMENTARIOS',
        ).select(
                'data_extracao',
                'DATA_PUBLICACAO',
                'ID_CANAL',
                'NM_CANAL',
                'ID_CATEGORIA',
                'ID_VIDEO',
                'TITULO_VIDEO',
                'DESCRICAO',
                'TAGS',
                'DURACAO_VIDEOS',
                'TOTAL_VISUALIZACOES',
                'TOTAL_LIKES',
                'TOTAL_FAVORITOS',
                f.explode('TOTAL_COMENTARIOS').alias('TOTAL_COMENTARIOS'),
        )
    dataframe_transform  = dataframe_transform.na.drop('all')
    dataframe_transform = dataframe_transform.withColumn("data_hora_extracao", f.col('data_extracao'))
    dataframe_transform = dataframe_transform.withColumn("data_extracao", f.date_format(f.unix_timestamp("data_extracao", "yyyy-MM-dd HH:mm:ss").cast("timestamp"), "yyyy-MM-dd"))
    dataframe_transform = dataframe_transform.withColumn('DURACAO_VIDEO_SEGUNDOS', converter_segundos(f.col('DURACAO_VIDEOS')))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_VISUALIZACOES', f.col('TOTAL_VISUALIZACOES').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_LIKES', f.col('TOTAL_LIKES').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_FAVORITOS', f.col('TOTAL_FAVORITOS').cast('int'))
    dataframe_transform = dataframe_transform.withColumn('TOTAL_COMENTARIOS', f.col('TOTAL_COMENTARIOS').cast('int'))
    return dataframe_transform
    

In [31]:
df_estatisticas = carregar_dataframe_prata(spark_session=spark, assunto='assunto_cities_skylines', metrica='estatisticas')
df_estatisticas = transformacao_dataframe(df_estatisticas)
df_estatisticas.show(truncate=False)

+-------------+--------------------+------------------------+----------------------------+------------+-----------+------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

<p>Depara Canal</p>

In [32]:
df_depara_canal = df_estatisticas.select(
    df_estatisticas.ID_CANAL,
    df_estatisticas.NM_CANAL
).distinct()

df_depara_canal.show(truncate=False)

+------------------------+-------------------------------------+
|ID_CANAL                |NM_CANAL                             |
+------------------------+-------------------------------------+
|UCvEj9EurcC8q5hkV-Apf4jA|Sr L                                 |
|UCMsbzwrSvRJoa0JhS5YcsqQ|rlkpedrom                            |
|UCRt5pfs_8hq3-GUSjULoV9Q|Team Random                          |
|UCIv5D2ro1owj6P7OdZ4-BiA|BL4D3N | Jogos de Gestão e Construção|
|UCrOH1V-FyMunBIMrKL0y0xQ|ChratosGameplay                      |
|UCYEb5mbKZ5JJzN0xBuKyaig|Canal Sala de Jogos                  |
|UClGA2euv_zdZ442GcVBjcbg|Taverna Replicante                   |
|UC7v51W2NoLFawizgcNuVBOA|CANAL HAND - GAME WITH MUSIC         |
|UCHEuAFdws9OCZPcfTBuxtCw|Migeez                               |
|UCGved3ynk6ZhUvSKxzgweTw|Jogando                              |
|UCcn60fs5EMW8A_k9R-e4gEg|Cabrunco Games                       |
|UCOYJm7BsLV8VCkW2DC1MI5w|Lucas gameplays brasil               |
|UCLXbP3xz78Yz7HW5avm4zDg

<p>Depara Vídeo</p>

In [33]:
df_depara_video = df_estatisticas.select(
    df_estatisticas.ID_VIDEO,
    df_estatisticas.TITULO_VIDEO
).distinct()

df_depara_video.show(truncate=False)


+-----------+----------------------------------------------------------------------------------------------+
|ID_VIDEO   |TITULO_VIDEO                                                                                  |
+-----------+----------------------------------------------------------------------------------------------+
|zS3t3k4ZtU8|O Sucesso de CITIES SKYLINES Explicado em 5 minutos!                                          |
|ge2WfHGhYwQ|CONSTRUINDO A AREA DE FAZENDA - Rio de fevereiro - Cities Skylines #9                         |
|SQg_cqgyDLg|APRENDENDO A JOGAR CITIES SKYLINES 2!  #01                                                    |
|TqDfKIao-1Q|Nada melhor que um trabalho de escritório!  | Cities Skylines 2 #07                           |
|aluyojsJljE|Cities: Skylines II Análises e dicas                                                          |
|ANa7la0wqNc|A Volta do Melhor CONSTRUTOR DE CIDADES do Mundo - Cities Skylines 2 [Gameplay PT-BR]         |
|wCLSZxLfUAk|MELHOR

In [34]:
df_estatisticas.printSchema()

root
 |-- data_extracao: string (nullable = true)
 |-- DATA_PUBLICACAO: string (nullable = true)
 |-- ID_CANAL: string (nullable = true)
 |-- NM_CANAL: string (nullable = true)
 |-- ID_CATEGORIA: string (nullable = true)
 |-- ID_VIDEO: string (nullable = true)
 |-- TITULO_VIDEO: string (nullable = true)
 |-- DESCRICAO: string (nullable = true)
 |-- TAGS: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- DURACAO_VIDEOS: string (nullable = true)
 |-- TOTAL_VISUALIZACOES: integer (nullable = true)
 |-- TOTAL_LIKES: integer (nullable = true)
 |-- TOTAL_FAVORITOS: integer (nullable = true)
 |-- TOTAL_COMENTARIOS: integer (nullable = true)
 |-- data_hora_extracao: string (nullable = true)
 |-- DURACAO_VIDEO_SEGUNDOS: integer (nullable = true)



<p>O Número de comentários e likes gera mais views? </p>

In [69]:
def gerar_relacao_like_comentario_view_favoritos(dataframe: SparkSession):
    df_comentarios_likes = dataframe.select(
        df_estatisticas.data_hora_extracao,
        df_estatisticas.data_extracao,
        df_estatisticas.ID_CANAL,
        df_estatisticas.NM_CANAL,
        df_estatisticas.ID_VIDEO,
        df_estatisticas.TITULO_VIDEO,
        df_estatisticas.TOTAL_VISUALIZACOES,
        df_estatisticas.TOTAL_LIKES,
        df_estatisticas.TOTAL_FAVORITOS,
        df_estatisticas.TOTAL_COMENTARIOS
    ).filter(
        # (df_estatisticas.data_extracao == '2023-10-27') &
        (dataframe.TOTAL_VISUALIZACOES > 0) &
        (dataframe.TOTAL_LIKES > 0) 
    ).sort(f.col('TOTAL_VISUALIZACOES').desc())
    return df_comentarios_likes


In [70]:
df_comentarios_likes = df_estatisticas.select(
    df_estatisticas.data_hora_extracao,
    df_estatisticas.data_extracao,
    df_estatisticas.ID_CANAL,
    df_estatisticas.NM_CANAL,
    df_estatisticas.ID_VIDEO,
    df_estatisticas.TITULO_VIDEO,
    df_estatisticas.TOTAL_VISUALIZACOES,
    df_estatisticas.TOTAL_LIKES,
    df_estatisticas.TOTAL_FAVORITOS,
    df_estatisticas.TOTAL_COMENTARIOS
).filter(
    # (df_estatisticas.data_extracao == '2023-10-27') &
    (df_estatisticas.TOTAL_VISUALIZACOES > 0) &
    (df_estatisticas.TOTAL_LIKES > 0) 
).sort(f.col('TOTAL_VISUALIZACOES').desc())
df_comentarios_likes.show(truncate=False)

+-------------------+-------------+------------------------+---------------+-----------+----------------------------------------------------------------------------+-------------------+-----------+---------------+-----------------+
|data_hora_extracao |data_extracao|ID_CANAL                |NM_CANAL       |ID_VIDEO   |TITULO_VIDEO                                                                |TOTAL_VISUALIZACOES|TOTAL_LIKES|TOTAL_FAVORITOS|TOTAL_COMENTARIOS|
+-------------------+-------------+------------------------+---------------+-----------+----------------------------------------------------------------------------+-------------------+-----------+---------------+-----------------+
|2023-10-27 19:01:50|2023-10-27   |UCrOH1V-FyMunBIMrKL0y0xQ|ChratosGameplay|wCLSZxLfUAk|MELHOR JOGO DE CONSTRUIR CIDADES CHEGOU E ESTÁ LINDO 🏙️ - Cities Skylines 2|120812             |13988      |0              |418              |
|2023-10-27 18:01:34|2023-10-27   |UCrOH1V-FyMunBIMrKL0y0xQ|ChratosGamepl

In [72]:
df_comentarios_likes = gerar_relacao_like_comentario_view_favoritos(df_estatisticas)
df_comentarios_likes.show()

+-------------------+-------------+--------------------+---------------+-----------+--------------------+-------------------+-----------+---------------+-----------------+
| data_hora_extracao|data_extracao|            ID_CANAL|       NM_CANAL|   ID_VIDEO|        TITULO_VIDEO|TOTAL_VISUALIZACOES|TOTAL_LIKES|TOTAL_FAVORITOS|TOTAL_COMENTARIOS|
+-------------------+-------------+--------------------+---------------+-----------+--------------------+-------------------+-----------+---------------+-----------------+
|2023-10-27 19:01:50|   2023-10-27|UCrOH1V-FyMunBIMr...|ChratosGameplay|wCLSZxLfUAk|MELHOR JOGO DE CO...|             120812|      13988|              0|              418|
|2023-10-27 18:01:34|   2023-10-27|UCrOH1V-FyMunBIMr...|ChratosGameplay|wCLSZxLfUAk|MELHOR JOGO DE CO...|             120581|      13975|              0|              418|
|2023-10-27 17:00:46|   2023-10-27|UCrOH1V-FyMunBIMr...|ChratosGameplay|wCLSZxLfUAk|MELHOR JOGO DE CO...|             120404|      13965|   

In [79]:
criar_particao_ouro(dataframe=df_comentarios_likes, 
                    assunto_pesquisa='assunto_cities_skylines', 
    metrica='relacao_viusalicacao_comentario_likes',
    colunas_particao=('data_extracao', 'ID_CANAL', 'ID_VIDEO'),
    nome_arquivo='relacao_viusalicacao_comentario_likes.parquet',
)